<a href="https://colab.research.google.com/github/sappyb/Codes-Doc/blob/master/Model_flow_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.model_selection import train_test_split
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
images_dir = '/content/gdrive/My Drive/rate_distribution'
model_dir = '/content/gdrive/My Drive/rate_distribution/mymodel_4'
work_dir = '/content/gdrive/My Drive/rate_distribution'

In [ ]:
applications = ['2d', '3d', '4d', 'nekbone', 'lammps', 'milc']
ranks = ['1024']

In [ ]:
prepivot_train = pd.DataFrame()
prepivot_test = pd.DataFrame()
number_of_flows_per_application = []
for i in applications:
  for j in ranks:
    f_name = '/content/gdrive/My Drive/csvs/full-{}-{}.csv'.format(i, j)
    print(f_name)
    data_a = pd.read_csv(f_name, on_bad_lines='skip')
    data_a = data_a.sample(frac=1).reset_index(drop=True)
    print(i)
    print(j)
    print(data_a.shape)

    if j == '32':
      if i == 'amg':
        data_a = data_a.head(int(100 * 3.5))
      elif i == 'kripke':
        data_a = data_a.head(int(100 * 4))
        data_a = pd.DataFrame(pd.np.repeat(data_a.values,2,axis=0),columns=data_a.columns)
      elif i == 'laghos':
        data_a = data_a.head(int(100 * 6))
      else:
        data_a = data_a.head(int(100))
    if j == '64':
      if i == 'amg':
        data_a = data_a.head(int(220 * 3.5))
      elif i == 'kripke':
        data_a = data_a.head(int(220 * 4))
        data_a = pd.DataFrame(pd.np.repeat(data_a.values,2,axis=0),columns=data_a.columns)
      elif i == 'laghos':
        data_a = data_a.head(int(220 * 6))
      else:
        data_a = data_a.head(int(220))
    if j == '128':
      if i == 'amg':
        data_a = data_a.head(int(460 * 3.5))
      if i == 'kripke':
        data_a = data_a.head(int(460 * 4))
        data_a = pd.DataFrame(pd.np.repeat(data_a.values,2,axis=0),columns=data_a.columns)
      if i == 'laghos':
        data_a = data_a.head(int(460 * 6))
      else:
        data_a = data_a.head(int(460))
    if j == '256':
      if i == 'amg':
        data_a = data_a.head(int(950 * 3.5))
      elif i == 'kripke':
        data_a = data_a.head(int(950 * 4))
        data_a = pd.DataFrame(pd.np.repeat(data_a.values,2,axis=0),columns=data_a.columns)
      elif i == 'laghos':
        data_a = data_a.head(int(950 * 6))
      else:
        data_a = data_a.head(int(950))
    if j == '512':
      if i == 'amg':
        data_a = data_a.head(int(1950 * 3.5))
      elif i == 'kripke':
        data_a = data_a.head(int(1950 * 4))
        data_a = pd.DataFrame(pd.np.repeat(data_a.values,2,axis=0),columns=data_a.columns)
      elif i == 'laghos':
        data_a = data_a.head(int(1950 * 6))
      else:
        data_a = data_a.head(int(1950))
    if j == '1024':
      data_a = data_a.head(int(3900))
      print(data_a.shape)

    data_list = []
    for k in range(1, 246):
      data_list.append('Data_{}'.format(k))
    data_a['result'] = data_a[data_list].sum(axis=1)
    print(data_a['result'])
    elephant = []
    for z in data_a['result'].to_list():
        if z > (1 * 10**7):
          elephant.append(1)
        else:
          elephant.append(0)
    data_a['Is_Elephant'] = elephant
    data_a['Application'] = i
    data_a['Rank'] = j
    prepivot_train = pd.concat([prepivot_train, data_a])
    number_of_flows_per_application.append(data_a.shape[0])
    elephant.clear()
print(prepivot_train)



/content/gdrive/My Drive/csvs/full-2d-1024.csv
2d
1024
(4096, 301)
(3900, 301)
0       77930752
1       77963456
2       77996352
3       77932736
4       77922368
          ...   
3895    77930432
3896    77851584
3897    77963392
3898    78029312
3899    77930560
Name: result, Length: 3900, dtype: int64
/content/gdrive/My Drive/csvs/full-3d-1024.csv
3d
1024
(6144, 301)
(3900, 301)
0       57495040
1       57502272
2       57590720
3       57568000
4       57502272
          ...   
3895    57582336
3896    57534976
3897    57535168
3898    57568000
3899    57568000
Name: result, Length: 3900, dtype: int64
/content/gdrive/My Drive/csvs/full-4d-1024.csv
4d
1024
(8192, 301)
(3900, 301)
0       50988928
1       51021824
2       51021632
3       50988800
4       51053568
          ...   
3895    51036096
3896    51007424
3897    51021824
3898    51021568
3899    51054592
Name: result, Length: 3900, dtype: int64
/content/gdrive/My Drive/csvs/full-nekbone-1024.csv
nekbone
1024
(5709, 246)
(3

In [ ]:
prepivot_train = prepivot_train.fillna(0)

In [ ]:
num_of_samples_train = prepivot_train.shape[0]
num_of_time_steps = 301
num_of_features = 1

In [ ]:
labels_train = prepivot_train['Is_Elephant'].to_list()

In [ ]:
for i in range(301):
  prepivot_train = prepivot_train.drop('Time_{}'.format(i), axis='columns')
  prepivot_train = prepivot_train.drop('Rate_{}'.format(i), axis='columns')
  prepivot_train = prepivot_train.drop('Packet_Rate_{}'.format(i), axis='columns')

KeyError: "['Time_0'] not found in axis"

In [ ]:
prepivot_train.iloc[:, 1:-4]

,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Data_8,Data_9,Data_10,...,Data_291,Data_292,Data_293,Data_294,Data_295,Data_296,Data_297,Data_298,Data_299,Data_300
0,328896,263296,328960,328896,328896,296192,328960,328960,328960,295936,...,328896.0,329024.0,328960.0,321664.0,303360.0,296064.0,328960.0,296000.0,328896.0,329088.0
1,296128,296064,328960,296128,328960,361728,296192,328832,296128,329024,...,328960.0,328896.0,328896.0,296192.0,328896.0,296128.0,328896.0,329024.0,361728.0,329088.0
2,296000,296128,328960,328896,296128,328896,342336,315584,328960,296128,...,296192.0,328960.0,328960.0,296064.0,328960.0,299136.0,325888.0,361728.0,328960.0,343424.0
3,298240,296064,328896,314560,310528,328960,328896,329024,296064,328960,...,329024.0,328896.0,328960.0,296128.0,328960.0,296000.0,296128.0,328960.0,328960.0,328960.0
4,361792,263168,328960,296128,328896,328960,328960,296128,361792,296064,...,321728.0,309504.0,322816.0,328960.0,312448.0,312576.0,328896.0,296064.0,296128.0,328896.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,219136,278656,328704,169104,0,0,497792,230400,267408,0,...,0.0,0.0,497728.0,141376.0,356496.0,0.0,0.0,0.0,0.0,0.0
3896,139264,417920,306176,132224,0,87040,410752,306176,191616,0,...,0.0,0.0,497792.0,249856.0,247936.0,0.0,0.0,0.0,0.0,0.0
3897,330752,167040,328704,169088,0,260096,237696,236544,261248,0,...,0.0,200704.0,297024.0,155712.0,342144.0,0.0,121856.0,0.0,0.0,0.0
3898,178176,178304,194560,125064,266248,201728,162944,192512,172168,250888,...,216072.0,223232.0,123008.0,223232.0,210056.0,137224.0,271360.0,0.0,0.0,0.0


In [ ]:
prepivot_train.iloc[:, :]

,FlowID,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Data_8,Data_9,...,Data_295,Data_296,Data_297,Data_298,Data_299,Data_300,result,Is_Elephant,Application,Rank
0,972,328896,263296,328960,328896,328896,296192,328960,328960,328960,...,303360.0,296064.0,328960.0,296000.0,328896.0,329088.0,77930752,1,2d,1024
1,3687,296128,296064,328960,296128,328960,361728,296192,328832,296128,...,328896.0,296128.0,328896.0,329024.0,361728.0,329088.0,77963456,1,2d,1024
2,1350,296000,296128,328960,328896,296128,328896,342336,315584,328960,...,328960.0,299136.0,325888.0,361728.0,328960.0,343424.0,77996352,1,2d,1024
3,3506,298240,296064,328896,314560,310528,328960,328896,329024,296064,...,328960.0,296000.0,296128.0,328960.0,328960.0,328960.0,77932736,1,2d,1024
4,2731,361792,263168,328960,296128,328896,328960,328960,296128,361792,...,312448.0,312576.0,328896.0,296064.0,296128.0,328896.0,77922368,1,2d,1024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,0,219136,278656,328704,169104,0,0,497792,230400,267408,...,356496.0,0.0,0.0,0.0,0.0,0.0,46793200,1,milc,1024
3896,0,139264,417920,306176,132224,0,87040,410752,306176,191616,...,247936.0,0.0,0.0,0.0,0.0,0.0,46792448,1,milc,1024
3897,0,330752,167040,328704,169088,0,260096,237696,236544,261248,...,342144.0,0.0,121856.0,0.0,0.0,0.0,46841600,1,milc,1024
3898,319488,178176,178304,194560,125064,266248,201728,162944,192512,172168,...,210056.0,137224.0,271360.0,0.0,0.0,0.0,46836208,1,milc,1024


In [ ]:
data_test_probe = np.array(prepivot_train.iloc[:, 1:-4])

In [ ]:
row_with_max_value = np.amax(data_test_probe, axis=0)
row_with_max_value

array([497792., 673920., 361792., 361856., 475136., 497728., 676992.,
       435328., 372096., 361856., 497792., 655488., 754816., 470095.,
       389120., 497792., 579712., 729216., 446464., 361856., 497728.,
       505984., 710784., 408576., 394624., 497664., 497792., 695424.,
       361856., 374144., 497664., 497728., 637056., 756864., 361856.,
       381952., 497792., 612480., 750720., 406656., 361856., 497728.,
       562304., 720000., 446464., 361856., 497728., 500864., 704640.,
       398336., 361856., 497664., 497792., 695424., 361792., 361856.,
       435200., 497792., 649344., 741504., 361856., 365568., 497792.,
       614528., 734336., 375944., 371712., 497792., 544896., 710784.,
       446464., 361856., 497664., 497792., 690304., 376832., 368000.,
       497664., 497792., 685184., 361856., 390592., 368000., 497792.,
       647296., 763008., 460596., 379904., 497792., 611456., 748672.,
       407688., 370048., 497728., 562304., 721024., 446464., 361856.,
       488448., 4977

In [ ]:
np.sum(row_with_max_value) // 300

502293.0

In [ ]:
np.amax(data_test_probe)

763008.0

In [ ]:
np.amax(np.array(prepivot_train['result']))

78095040

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
xp_train = scaler.fit_transform(prepivot_train.iloc[:, 2:-4])
#xp_train = prepivot_train.iloc[:, 2:-4]

Arranging the data from 2D to 3D,

X axis = Sample
Y axis = Time_step
Z axis = Features


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.preprocessing import sequence
from keras.regularizers import l2
from keras import optimizers
import tensorflow as tf

In [ ]:
epochs = 50
batch_size = 1024
time_steps = 301

In [ ]:
labels_1_train = [[i] for i in labels_train]
labels_1_train = np.array(labels_1_train)

In [ ]:
train_X = xp_train
train_Y = labels_1_train

In [ ]:
model = Sequential()
#model.add(Dense(1, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
hist = model.fit(train_X, train_Y, validation_split=0.2, epochs=epochs, batch_size=batch_size)
print(model.summary())
model.save('model.keras')

Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.6573 - loss: 0.4745 - val_accuracy: 0.9391 - val_loss: 0.3990
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9655 - loss: 0.2300 - val_accuracy: 0.9985 - val_loss: 0.3476
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9991 - loss: 0.2062 - val_accuracy: 1.0000 - val_loss: 0.3027
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 0.1772 - val_accuracy: 1.0000 - val_loss: 0.2649
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 0.1564 - val_accuracy: 1.0000 - val_loss: 0.2329
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 0.1374 - val_accuracy: 1.0000 - val_loss: 0.2059
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.1223 - val_accuracy: 1.0000 - val_loss: 0.1829
Epoch 8/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 1.0000 - loss: 0.1086 - val_accuracy: 1.0000 - v

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                      │ (None, 1)                   │             300 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 903 (3.53 KB)

 Trainable params: 300 (1.17 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 603 (2.36 KB)

None


Testing with other data, incomplete


In [ ]:
import tensorflow as tf

In [ ]:
#model = tf.keras.models.load_model(model_dir)
#applications = ['subcomm3d']
#ranks = ['32', '64', '128', '256', '512']
app = []
accuracy = []
count = 0
current = 0
print(number_of_flows_per_application)
for i in applications:
  for j in ranks:
    label_test = prepivot_train.iloc[current:current+number_of_flows_per_application[count], :]['Is_Elephant'].to_list()
    dataq_test = xp_train
    labels_1_test = [[i] for i in label_test]
    labels_1_test = np.array(labels_1_test)
    test_X_1 = dataq_test[current:current+number_of_flows_per_application[count], :]
    test_Y_1 = labels_1_train[current:current+number_of_flows_per_application[count], :]
    scores = model.evaluate(test_X_1, test_Y_1, verbose=0)
    predict= model.predict(test_X_1, verbose=1)
    print(predict.shape)
    app.append('{}-{}'.format(i, j))
    accuracy.append(scores[1]*100)
    print("Accuracy of {} {}: {}".format(i, j, scores[1]*100))
    current += number_of_flows_per_application[count]
    count += 1

[3900, 3900, 3900, 3900, 3900, 3900]
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
(3900, 1)
Accuracy of 2d 1024: 100.0
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
(3900, 1)
Accuracy of 3d 1024: 100.0
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
(3900, 1)
Accuracy of 4d 1024: 100.0
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
(3900, 1)
Accuracy of nekbone 1024: 100.0
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
(3900, 1)
Accuracy of lammps 1024: 100.0
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
(3900, 1)
Accuracy of milc 1024: 100.0


In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model(model_dir)

In [ ]:
applications = ['laghos']
ranks = ['32', '64', '128', '256', '512']
#applications = ['swfft']
ranks = ['32','64', '128', '256', '512']
#applications = ['amg', 'kripke', 'laghos', 'stencil4d', 'subcomm3d', 'sw4lite', 'minife']
#ranks = ['32', '64', '128', '256', '512']

In [ ]:
for a in applications:
  for r in ranks:
    #f_name = '/content/gdrive/My Drive/csvstest/full-{}-{}.csv'.format(a, r)
    f_name = '/content/gdrive/My Drive/csvs/full-{}-{}.csv'.format(a, r)
    print(f_name)
    data_a = pd.read_csv(f_name)
    data_a = data_a.sample(frac=1).reset_index(drop=True)
    data_list = []
    range_of_columns = 301
    for k in range(300):
      data_list.append('Data_{}'.format(k))
    data_a['result'] = data_a[data_list].sum(axis=1)
    elephant = []
    for z in data_a['result'].to_list():
        if z > (1 * 10**8):
          elephant.append(1)
        else:
          elephant.append(0)
    data_a['Is_Elephant'] = elephant
    data_a = data_a.fillna(0)
    labels_train = data_a['Is_Elephant'].to_list()
    #print(data_a)
    #print(labels_train)
    for k in range(301):
      if 'Time_{}'.format(k) in data_a.columns:
        data_a = data_a.drop('Time_{}'.format(k), axis='columns')
      if 'Rate_{}'.format(k) in data_a.columns:
        data_a = data_a.drop('Rate_{}'.format(k), axis='columns')
      if 'Packet_Rate_{}'.format(k) in data_a.columns:
        data_a = data_a.drop('Packet_Rate_{}'.format(k), axis='columns')
    result = pd.DataFrame()
    shape_of_df = data_a.iloc[:, 2:8].shape[1] # Current shape of dataframe
    #print(shape_of_df)
    number_of_times = range_of_columns // shape_of_df
    number_of_add = range_of_columns % shape_of_df
    for ii in range(number_of_times):
      result = pd.concat([result, data_a.iloc[:, 2:(shape_of_df+2)]], axis=1)
    result = pd.concat([result, data_a.iloc[:, 2:(number_of_add+2)]], axis=1)
    #print(shape_of_df, number_of_times)
    #print(result.shape)
    #print(result)
    scaler = MinMaxScaler(feature_range=(0,1))
    xp_train = scaler.fit_transform(result)

    labels_1_train = [[i] for i in labels_train]
    labels_1_train = np.array(labels_1_train)
    train_X = xp_train
    train_Y = labels_1_train
    scores = model.evaluate(train_X, train_Y, verbose=0)
    predict= model.predict(train_X, verbose=1)
    #print(predict)
    print("Accuracy of {} {}: {}".format(a, r , scores[1]*100))

/content/gdrive/My Drive/csvs/full-laghos-32.csv
9/9 [==============================] - 0s 2ms/step
Accuracy of laghos 32: 24.372759461402893
/content/gdrive/My Drive/csvs/full-laghos-64.csv
21/21 [==============================] - 0s 2ms/step
Accuracy of laghos 64: 56.711918115615845
/content/gdrive/My Drive/csvs/full-laghos-128.csv
46/46 [==============================] - 0s 1ms/step
Accuracy of laghos 128: 34.83068346977234
/content/gdrive/My Drive/csvs/full-laghos-256.csv
98/98 [==============================] - 0s 1ms/step
Accuracy of laghos 256: 25.07195472717285
/content/gdrive/My Drive/csvs/full-laghos-512.csv
211/211 [==============================] - 0s 1ms/step
Accuracy of laghos 512: 60.05648970603943


In [ ]:
print(train_X.shape)

(51221, 301)


In [ ]:

!saved_model_cli show --dir model --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['dense_1_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 301)
      name: serving_default_dense_1_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [ ]:
from google.colab import files
!zip -r model.zip model
files.download("model.zip")

  adding: model/ (stored 0%)
  adding: model/variables/ (stored 0%)
  adding: model/variables/variables.index (deflated 55%)
  adding: model/variables/variables.data-00000-of-00001 (deflated 32%)
  adding: model/saved_model.pb (deflated 85%)
  adding: model/keras_metadata.pb (deflated 78%)
  adding: model/assets/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("test_num.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(train_X)
np.savetxt('test_num.txt', train_X, fmt='%f')


[[3.67508196e-02 4.63011706e-02 3.80958194e-02 ... 3.26176514e-02
  3.87473501e-02 1.10198917e-01]
 [0.00000000e+00 3.17889014e-05 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.39453110e-05 2.59333857e-03 4.44408841e-04 ... 3.99917377e-04
  4.90472786e-04 1.28737053e-03]
 ...
 [1.50238092e-04 1.62759175e-04 8.13795875e-05 ... 7.34425026e-05
  1.60563981e-04 4.70809793e-04]
 [9.61758537e-03 1.04191303e-02 5.20956515e-03 ... 4.70146770e-03
  1.02786036e-02 3.01391831e-02]
 [9.61758537e-03 1.04191303e-02 5.20956515e-03 ... 4.70146770e-03
  1.02786036e-02 3.01391831e-02]]


In [ ]:
predict= model.predict(train_X, verbose=1)
print(predict)

1601/1601 [==============================] - 2s 1ms/step
[[0.9857718 ]
 [0.00612363]
 [0.0072327 ]
 ...
 [0.00630432]
 [0.04248509]
 [0.04241738]]
